In [ ]:
using Distributions, Random, Graphs, SparseArrays, StatsBase, DataFrames, Statistics

In [ ]:
function Communication_subgraph(G, p)
    CG = copy(G)
    for e in edges(G)
        ok = rand(Bernoulli(p))
        if ok == 0
            rem_edge!(CG, e)
        end
    end
    return CG
end

# Distance Based Model

In [ ]:
function Distance_Based_Diffusion(G::Graph, S1, S2, p, π)
    Cs = Communication_subgraph(G, p)
    
    intersection = intersect(S1,S2) 
    pi = rand(Bernoulli(π),length(intersection))
    filter!(x->x∉(1 .- pi).*intersection,S1)
    filter!(x->x∉(pi).*intersection,S2)
    
    Dist1 = bellman_ford_shortest_paths(Cs, S1).dists
    Dist2 = bellman_ford_shortest_paths(Cs, S2).dists
    
    Activ1 = Dist2 .> Dist1
    Activ2 = Dist1 .> Dist2
    Activ = ifelse.(((Dist1 .> nv(G)) .& (Dist2 .> nv(G))) .== 1, 0, 1)
    
    return sum(Activ1)/nv(G), sum(Activ2)/nv(G), sum(Activ)/nv(G)
end

# Wave Propagation Model

In [ ]:
function Wave_Propagation_Diffusion(G::Graph, S1, S2, p)
    Cs = Communication_subgraph(G, p)
    Pr1 = vec(zeros(nv(G),1))
    Pr2 = vec(zeros(nv(G),1))
    Pr1[S1] .= 1
    Pr2[S2] .= 1
    
    Dist1 = bellman_ford_shortest_paths(Cs, S1).dists
    Dist2 = bellman_ford_shortest_paths(Cs, S2).dists
    Dist = bellman_ford_shortest_paths(Cs, vcat(S1,S2)).dists
    Dim = maximum(filter(x->x<nv(G),Dist))
    
    for d in 1:Dim
        temp1 = []
        temp2 = []
        vlist = findall(x->x==d, Dist)
        for v in vlist
            ns1 = Pr1[neighbors(Cs, v)]
            ns2 = Pr2[neighbors(Cs, v)]
            push!(temp1,sum(ns1)/length(findall(x->x!=0, ns1+ns2)))
            push!(temp2,sum(ns2)/length(findall(x->x!=0, ns1+ns2)))
        end
        Pr1[vlist] .= temp1
        Pr2[vlist] .= temp2
    end
    return sum(Pr1)/nv(G), sum(Pr2)/nv(G)
end

# Competitive Cascade Model

In [ ]:
function competitive_cascade(G::Graph, S1, S2, p, π)
    n = nv(G)
    Cs = Communication_subgraph(G, p)
    intersection = intersect(S1,S2)
    pi = rand(Bernoulli(π),length(intersection))
    filter!(x->x∉(1 .- pi).*intersection,S1)
    filter!(x->x∉(pi).*intersection,S2)
    lately_informed1 = copy(S1)
    lately_informed2 = copy(S2)
    Dist = bellman_ford_shortest_paths(Cs, vcat(S1,S2)).dists
    Dim = maximum(filter(x -> x<n, Dist))
    informed1 = copy(S1)
    informed2 = copy(S2)
    tree = zeros(Dim+1, n)
    tree[1, S1] .= 1
    tree[1, S2] .= 2
    for l in 1:Dim
        Dist = bellman_ford_shortest_paths(Cs, lately_informed1).dists
        new_informed1 = setdiff(findall(==(1), Dist), union(informed1,informed2))
        Dist = bellman_ford_shortest_paths(Cs, lately_informed2).dists
        new_informed2 = setdiff(findall(==(1), Dist), union(informed1,informed2))
        intersection = intersect(new_informed1, new_informed2)
        pi = rand(Bernoulli(π),length(intersection))
        filter!(x->x∉(1 .- pi).*intersection, new_informed1)
        filter!(x->x∉(pi).*intersection, new_informed2)
        tree[l+1, new_informed1] .= 1
        tree[l+1, new_informed2] .= 2
        lately_informed1 = copy(new_informed1)
        lately_informed2 = copy(new_informed2)
        informed1 = union(informed1, lately_informed1)
        informed2 = union(informed2, lately_informed2)
    end
    return length(informed1)/n, length(informed2)/n
end